In [31]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.pairwise import pairwise_distances

In [32]:
df = pd.read_csv('breast_cancer_coimbra_train.csv')

In [33]:
import numpy as np
import random
import operator
from copy import deepcopy

# Define the operations allowed in the trees
OPERATORS = {
    'add': operator.add,
    'sub': operator.sub,
    'mul': operator.mul,
    'div': operator.truediv  # safe division
}

TERMINALS = df.columns[:-1]  # Your dataset's features
TERMINALS


# Node class to represent tree nodes (either operator or terminal)
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def evaluate(self, features):
        """ Recursively evaluate the expression tree based on feature values in a DataFrame row """
        if isinstance(self.value, str):
            if self.value in OPERATORS:
                # Recursively evaluate left and right subtrees
                left_val = self.left.evaluate(features)
                right_val = self.right.evaluate(features)
                return OPERATORS[self.value](left_val, right_val)
            else:
                # It's a terminal (feature), so we access the DataFrame row using the column name
                return features[self.value]  # Assumes 'features' is a pandas row
        else:
            # It's a constant value (e.g., a number)
            return self.value   # it's a constant



# Random Tree Initialization
def generate_random_tree(depth=3):
    if depth == 0:
        # Base case: return a terminal node
        terminal = random.choice(TERMINALS)
        return Node(terminal)
    else:
        # Recursive case: return an operator node with two subtrees
        operator = random.choice(list(OPERATORS.keys()))
        left_subtree = generate_random_tree(depth - 1)
        right_subtree = generate_random_tree(depth - 1)
        return Node(operator, left_subtree, right_subtree)


# Fitness function (mean squared error)
def fitness_function(individual, X, y):
    predictions = np.array(individual.evaluate(X.iloc[:]))
    predictions = np.nan_to_num(predictions, nan=0.0, posinf=0.0, neginf=0.0)
    predictions = predictions.reshape(-1,1)
    pairwise_d = pairwise_distances(predictions,predictions,metric='manhattan')
    clusters = AgglomerativeClustering(2).fit(pairwise_d)
    fitness = v_measure_score(clusters.labels_,y)
    return fitness


# Tournament Selection
def tournament_selection(population, fitnesses, tournament_size=3):
    selected = random.sample(list(zip(population, fitnesses)), tournament_size)
    return max(selected, key=lambda x: x[1])[0]  # Return the individual with the best fitness


# Crossover between two trees
def crossover(tree1, tree2, max_depth=3):
    if random.random() < 0.9:  # 70% chance to swap subtrees
        if isinstance(tree1.value, str) and tree1.value in OPERATORS:
            return Node(tree1.value, crossover(tree1.left, tree2.left, max_depth-1), crossover(tree1.right, tree2.right, max_depth-1))
        return deepcopy(tree2)
    else:
        return deepcopy(tree1)


# Mutation (replace a subtree with a random subtree)
def mutation(tree, max_depth=3):
    if random.random() < 0.1:  # 10% chance of mutation
        return generate_random_tree(max_depth)
    else:
        if isinstance(tree.value, str) and tree.value in OPERATORS:
            return Node(tree.value, mutation(tree.left, max_depth-1), mutation(tree.right, max_depth-1))
        return deepcopy(tree)

def print_tree(node, depth=0):
    """ Recursively prints the structure of the tree with indentation for clarity """
    indent = "  " * depth  # Indentation based on the depth of the node
    if node.left is None and node.right is None:
        # It's a terminal (leaf) node
        print(f"{indent}{node.value}")
    else:
        # It's an operator node
        print(f"{indent}{node.value}")
        # Recursively print left and right subtrees
        if node.left:
            print_tree(node.left, depth + 1)
        if node.right:
            print_tree(node.right, depth + 1)

# Evolutionary Algorithm (Genetic Programming)
def evolve_population(X, y, k,pop_size=100,clusters =2, generations=10, max_depth=3):
    # Step 1: Initialize the population
    population = [generate_random_tree(max_depth) for _ in range(pop_size)]

    for generation in range(generations):
        
        new_population = []
        # Step 2: Evaluate fitness
        fitnesses = [fitness_function(ind, X, y) for ind in population]

        sorted_with_indices = sorted(enumerate(fitnesses), key=lambda x: x[1],reverse=True)
        original_indices = [x[0] for x in sorted_with_indices]
        
        for idx in original_indices[:20]:
            new_population.append(population[idx])
            



        # Step 3: Selection and reproduction
        while len(new_population) < pop_size:
            # Tournament selection
            parent1 = tournament_selection(population, fitnesses,k)
            parent2 = tournament_selection(population, fitnesses,k)

            # Crossover
            if(random.random() < 0.9):
                child = crossover(parent1, parent2, max_depth)
            else:
            # Mutation
                child = mutation(parent1, max_depth)

            new_population.append(child)

        # Step 4: Replace the old population with the new population
        population = new_population

        # Print best fitness every generation
        best_fitness = max(fitnesses)
        median_fitness = np.mean(fitnesses)
        worst_fitness = min(fitnesses)
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
        print(f"Generation {generation + 1}, Median Fitness: {median_fitness}")
        print(f"Generation {generation + 1}, Worst Fitness: {worst_fitness}")
    # Return the best individual after evolution
    best_individual = 1
    return best_individual




In [34]:
import warnings
warnings.filterwarnings("ignore")

In [35]:
df = pd.read_csv('breast_cancer_coimbra_train.csv')

In [36]:
X = df.drop(columns=['Classification'])
y = df['Classification']
best_tree = evolve_population(X, y, 3,pop_size=500, generations=100, max_depth=5)
# print(f"Best individual: {best_tree}")
# print(f"Best fitness: {fitness_function(best_tree, X, y)}")

Generation 1, Best Fitness: 0.08415475742128092
Generation 1, Median Fitness: 0.019194843178714397
Generation 1, Worst Fitness: 1.113148687615448e-05
Generation 2, Best Fitness: 0.14164713571829352
Generation 2, Median Fitness: 0.024927969611218223
Generation 2, Worst Fitness: 5.514778287443675e-06
Generation 3, Best Fitness: 0.19387459982029692
Generation 3, Median Fitness: 0.0276523975001549
Generation 3, Worst Fitness: 5.514778287443675e-06
Generation 4, Best Fitness: 0.19387459982029692
Generation 4, Median Fitness: 0.02900807322644963
Generation 4, Worst Fitness: 5.514778287443675e-06
Generation 5, Best Fitness: 0.19387459982029692
Generation 5, Median Fitness: 0.03225362366569072
Generation 5, Worst Fitness: 2.5383490428103392e-05
Generation 6, Best Fitness: 0.19387459982029692
Generation 6, Median Fitness: 0.03580747660041871
Generation 6, Worst Fitness: 1.356957979045803e-05
Generation 7, Best Fitness: 0.19387459982029692
Generation 7, Median Fitness: 0.03680698718637902
Genera

KeyboardInterrupt: 

In [ ]:
a = generate_random_tree(2)
b = generate_random_tree(2)
print_tree(a)
print_tree(b)

sub
  sub
    Adiponectin
    Leptin
  add
    Adiponectin
    Leptin
mul
  sub
    Insulin
    Leptin
  mul
    Age
    Age


In [ ]:
print_tree(crossover(a,b))

sub
  sub
    Insulin
    Leptin
  add
    Age
    Age
